In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [ ]:

dataset_dir = "custom_sign_dataset"
os.makedirs(dataset_dir, exist_ok=True)
model_path = "gesture_model.h5"


In [ ]:

def record_gesture(label):
    cap = cv2.VideoCapture(0)
    image_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

   
        cv2.putText(frame, f"Recording for Label: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Press 'S' to Save, 'Q' to Quit", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow("Capture Gesture", frame)

        key = cv2.waitKey(1)
        if key & 0xFF == ord('s'): 
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (28, 28))
            filepath = os.path.join(dataset_dir, f"{label}_{image_count}.jpg")
            cv2.imwrite(filepath, resized)
            image_count += 1
            print(f"Saved: {filepath}")
        elif key & 0xFF == ord('q'):  
            break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Info", f"Recorded {image_count} images for label '{label}'.")

In [ ]:

def train_model():
  
    images, labels = [], []
    label_map = {}
    label_index = 0

    for filename in os.listdir(dataset_dir):
        if filename.endswith(".jpg"):
            filepath = os.path.join(dataset_dir, filename)
            label = filename.split('_')[0]

            
            if label not in label_map:
                label_map[label] = label_index
                label_index += 1

            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            images.append(img)
            labels.append(label_map[label])

    if not images:
        messagebox.showerror("Error", "No data found! Please record gestures first.")
        return

    images = np.array(images).reshape(-1, 28, 28, 1) / 255.0
    labels = np.array(labels)

  
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

   
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(2, 2),
        Dropout(0.2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(label_map), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    
    
    model.save(model_path)
    messagebox.showinfo("Info", "Model trained and saved successfully!")

In [ ]:

def predict_gesture():
    if not os.path.exists(model_path):
        messagebox.showerror("Error", "No trained model found! Train the model first.")
        return

    model = load_model(model_path)
    cap = cv2.VideoCapture(0)

    label_map = {v: k for k, v in enumerate(sorted(os.listdir(dataset_dir)))}

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (28, 28))
        input_data = np.expand_dims(resized, axis=(0, -1)) / 255.0

    
        predictions = model.predict(input_data)
        predicted_label_index = np.argmax(predictions)
        predicted_label = label_map.get(predicted_label_index, "Unknown")

       
        cv2.putText(frame, f'Prediction: {predicted_label}', (10, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Sign Language Prediction', frame)

      
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:

def start_gui():
    root = tk.Tk()
    root.title("Sign Language Translator")

    tk.Label(root, text="Enter Label for Gesture:").pack(pady=5)
    label_entry = tk.Entry(root)
    label_entry.pack(pady=5)

    tk.Button(root, text="Record Gesture", bg="lightblue",
              command=lambda: record_gesture(label_entry.get())).pack(pady=10)
    tk.Button(root, text="Train Model", bg="lightgreen", command=train_model).pack(pady=10)
    tk.Button(root, text="Start Prediction", bg="lightpink", command=predict_gesture).pack(pady=10)

    root.mainloop()

In [ ]:

start_gui()

Saved: custom_sign_dataset\4_0.jpg
Saved: custom_sign_dataset\4_1.jpg
Saved: custom_sign_dataset\4_2.jpg
Saved: custom_sign_dataset\4_3.jpg
Saved: custom_sign_dataset\4_4.jpg
Saved: custom_sign_dataset\4_5.jpg
Saved: custom_sign_dataset\4_6.jpg
Saved: custom_sign_dataset\4_7.jpg
Saved: custom_sign_dataset\4_8.jpg
Saved: custom_sign_dataset\4_9.jpg
Saved: custom_sign_dataset\4_10.jpg
Saved: custom_sign_dataset\4_11.jpg
Saved: custom_sign_dataset\4_12.jpg
Saved: custom_sign_dataset\4_13.jpg
Saved: custom_sign_dataset\4_14.jpg
Saved: custom_sign_dataset\4_15.jpg
Saved: custom_sign_dataset\4_16.jpg
Saved: custom_sign_dataset\4_17.jpg
Saved: custom_sign_dataset\4_18.jpg
Saved: custom_sign_dataset\4_19.jpg
Saved: custom_sign_dataset\4_20.jpg
Saved: custom_sign_dataset\4_21.jpg
Saved: custom_sign_dataset\4_22.jpg
Saved: custom_sign_dataset\4_23.jpg
Saved: custom_sign_dataset\4_24.jpg
Saved: custom_sign_dataset\4_25.jpg
Saved: custom_sign_dataset\4_26.jpg
Saved: custom_sign_dataset\4_27.jpg
Sa

c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.6534 - loss: 0.6369 - val_accuracy: 0.8696 - val_loss: 0.4182
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7273 - loss: 0.5141 - val_accuracy: 0.9565 - val_loss: 0.4307
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9247 - loss: 0.4131 - val_accuracy: 1.0000 - val_loss: 0.4053
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9055 - loss: 0.3442 - val_accuracy: 1.0000 - val_loss: 0.2231
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9229 - loss: 0.2409 - val_accuracy: 1.0000 - val_loss: 0.1798
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9826 - loss: 0.1717 - val_accuracy: 1.0000 - val_loss: 0.1104
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9904 - loss: 0.0884 - val_accuracy: 1.0000 - val_loss: 0.0321
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9826 - loss: 0.0587 - val_accuracy: 1.0000 - val_loss: 0.0267

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━